In [ ]:
import cv2
import numpy as np
import math
from random import randint

In [ ]:
from scipy.fft import fft2, fftshift

In [ ]:
def FFT_(image):    
    f = fft2(image)
    # Shift the zero-frequency component to the center
    fshift = fftshift(f)
    # Compute magnitude spectrum
    magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1e-8)  # add epsilon to avoid log(0)
    max_abs_val = np.max(np.abs(magnitude_spectrum))
    im = np.abs(magnitude_spectrum)/max_abs_val
    return im

In [ ]:
def crop_to_circle(image):
    """Crop the input image to a circular shape."""
    height, width = image.shape
    center = (width // 2, height // 2)
    radius = min(center)
    
    # Create a circular mask
    mask = np.zeros((height, width), dtype=np.uint8)
    cv2.circle(mask, center, radius, 255, -1)
    
    # Apply the mask to the image
    cropped = cv2.bitwise_and(image, image, mask=mask)
    return cropped, center, radius
def generate_nail_positions(center, radius, num_nails):
    """Generate evenly spaced nail positions around the circle."""
    nails = []
    for i in range(num_nails):
        angle = 2 * math.pi * i / num_nails
        x = int(center[0] + radius * math.cos(angle))
        y = int(center[1] + radius * math.sin(angle))
        nails.append((x, y))
    return nails


In [ ]:
def string_art_generator(image, num_nails=200, max_iterations=1000):
    canvas = np.ones((image.shape)) * 255
    cropped_image, center, radius = crop_to_circle(image)
    nail_positions = generate_nail_positions(center, radius, num_nails)
    # Start at a random nail
    current_nail_idx = randint(0, num_nails - 1)
    sequence = [current_nail_idx]
    visited_pairs = set()
    cropped_image = FFT_(cropped_image)
    canvasT = canvas
    # Main loop
    for _ in range(max_iterations - 1):
        current_nail = nail_positions[current_nail_idx]
        min_integral = float(np.inf)
        next_nail_idx = -1
        
        # Evaluate all possible next nails
        for i, next_nail in enumerate(nail_positions):
            if (i == current_nail_idx) or ((current_nail_idx, i) in visited_pairs):
                continue
            #integral,n = calculate_integral(cropped_image, current_nail, next_nail)
            canvasT = cv2.line(canvasT, current_nail, next_nail, (0,0,0), 1)
            tr = FFT_(canvasT)
            diff_ = (cropped_image - canvasT).mean().mean()
            if diff_ < min_integral:  
                min_integral = diff_
                next_nail_idx = i
        
        # If no valid next nail is found, break
        if next_nail_idx == -1:
            print("broken")
            break
        
        # Move to the next nail
        visited_pairs.add((current_nail_idx, next_nail_idx))
        canvas = cv2.line(canvas, current_nail, next_nail, (0,0,0), 1)
        current_nail_idx = next_nail_idx
        sequence.append(current_nail_idx)
    cv2.imshow('String Art',cropped_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return nail_positions, sequence


In [ ]:
image = cv2.imread('f1.jpg', cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (900,900))
#image = cv2.bitwise_not(image)
#_, image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV)
    
cv2.imshow('String Art', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#n = string_art_generator(image, num_nails=200, max_iterations=1000)